In [5]:
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb
import csv
from collections import defaultdict
import redis
import json
import time
import sklearn.utils
import psycopg2
from psycopg2.sql import Identifier, SQL
from datetime import datetime
import os

In [6]:
conn = redis.Redis(password='3oSYTdtZjsuSigRWLcG6VJt9gm4IMvYjQiqsSuGcAc-U4gMNpWGERAevXi9_SHNrn19piz7bBJG0iTLgx7DvknLHTECcHYrqmWb2rsuCWs89svKmhKDD_aMYaXq8IhSeg_89ooPZb0AqLRyR1-fa1zVjrh2UuV0sWFGSk5SjtW0', 
                   host='localhost', port=7380, decode_responses=True)
conn_psql_kongzi2 = psycopg2.connect(database='aida', user='zding', password='dingzishuo', host='localhost',
                                     port=5432)
conn_psql = psycopg2.connect(database='zding', user='zding', password='dingzishuo', host='localhost',
                                     port=5432)

In [7]:
def get_feature_results(exp_id, d_t, data_source='aida_conll'):
    fea_res = conn.lrange('result:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t), 0, -1)
    fea_vecs = [json.loads(res) for res in fea_res]
    return fea_vecs

def get_dataset_info(exp_id, d_t, data_source='aida_conll', can_size=50):
    valid_mens_size = conn.scard('valid_qry_ids:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    no_g_can_info_size = conn.scard('no_g_candidate_info:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    missed_g_candidate_size = conn.scard('missed_g_candidate:::::' + str(exp_id) + ':::::' + data_source + 
                                         ':::::' + str(d_t) + ':::::' + str(can_size))
    no_candidate_size = conn.scard('no_candidate:::::' + str(exp_id) + ':::::' + data_source + ':::::' + str(d_t))
    return valid_mens_size, no_candidate_size, no_g_can_info_size, missed_g_candidate_size

def fetch_all_features(exp_id, data_type, data_source='aida_conll'):
    res_feas = get_feature_results(exp_id, data_type, data_source)
    res_feas_ids = [[res[0].strip('(').split(', ')[0]] + res[1:-1] + [res[-1]] for res in res_feas]
    res_feas_ids = np.array(res_feas_ids, dtype=np.float64)
    return res_feas_ids

def fetch_all_features_delete_max_prior(exp_id, data_type, data_source='aida_conll'):
    res_feas = get_feature_results(exp_id, data_type, data_source)
    res_feas_ids = [[res[0].strip('(').split(', ')[0]] + res[1:2] + [res[2] if res[2]!=0 else res[1]] + res[3:] for res in res_feas]
    res_feas_ids = np.array(res_feas_ids, dtype=np.float64)
    return res_feas_ids

def trans_data(data):
    d_np = data[:, 1:-1]
    #print(d_np)
    d_labels = data[:, -1]
    #print(d_labels)
    idxs = np.where(d_labels == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_labels)) - idxs
    xgb_data = xgb.DMatrix(data=d_np, label=d_labels)
    xgb_data.set_group(d_groups)
    return xgb_data

def combine_features(original_feas, new_features):
    men_id_feas_dict = defaultdict(list)
    print("Building idx for new features...")
    for fea in new_features:
        men_id_feas_dict[fea[0]].append(fea)
    #for k, v in men_id_feas_dict.items():
    #    print(k)
    #    print(v)
    comb_feas = []
    pre_men_id = 0
    print("Combine original and new features...")
    for fea_idx, fea in enumerate(original_feas):
        #print('Processing: [%d] %s' % (fea_idx, fea))
        if pre_men_id == fea[0]:
            #print('skip')
            continue
        else:
            pre_men_id = fea[0]
            fea_size = len(men_id_feas_dict[fea[0]])
            res = np.append(original_feas[fea_idx: fea_idx + fea_size, :-1], np.array(men_id_feas_dict[fea[0]])[:, 1:], axis=1)
            #print('res:', res)
            comb_feas.append(res)
    return np.concatenate(comb_feas, axis=0)

def evalerror(preds, dt, d_tal_size):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    matched_ids = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        if(preds[q_id] == max(pre_res)):
            if len([x for x in pre_res if x == preds[q_id]]) == 1:
                matched_ids.append(q_id)
        q_id += x
    precision = float(len(matched_ids)) / len(d_groups)
    recall = float(len(matched_ids)) / d_tal_size
    f1 = 2 * precision * recall / (precision + recall)
    return len(matched_ids), precision, recall, f1

from collections import defaultdict
def evalerror_detail_log(preds, dt, d_tal_size):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
    correct_results = {}
    wrong_results = {}
    duplicates_results = {}
    group_info = {}
    matched_ids = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        if(preds[q_id] == max(pre_res)):
            correct_results[q_id] = pre_res
            if len([x for x in pre_res if x == preds[q_id]]) == 1:
                matched_ids.append(q_id)
            else:
                duplicates_results[q_id] = pre_res
        else:
            wrong_results[q_id] = pre_res
        q_id += x
    precision = float(len(matched_ids)) / len(d_groups)
    recall = float(len(matched_ids)) / d_tal_size
    f1 = 2 * precision * recall / (precision + recall)
    return len(matched_ids), precision, recall, f1, correct_results, wrong_results, duplicates_results

from collections import defaultdict
from ast import literal_eval
def get_groups_results(preds, dt, res_features):
    d_l = dt.get_label()
    idxs = np.where(d_l == 1)[0]
    d_groups = np.append(np.delete(idxs, 0), len(d_l)) - idxs
#     correct_results = {}
#     wrong_results = {}
#     duplicates_results = {}
#     group_info = {}
#     matched_ids = []
    correct_res_groups = []
    wrong_res_groups = []
    dup_res_groups = []
    q_id = 0
    for x in d_groups:
        pre_res = preds[q_id: x + q_id]
        pre_res_feas = res_features[q_id: x + q_id]
        pred_q_id, pred_ent = literal_eval(res_features[q_id+np.argmax(pre_res)][0])
        
        if(preds[q_id] == max(pre_res)):
            correct_res_groups.append([pred_q_id, pred_ent])
        else:
            wrong_res_groups.append([pred_q_id, pred_ent])
        q_id += x
#     precision = float(len(matched_ids)) / len(d_groups)
#     recall = float(len(matched_ids)) / d_tal_size
#     f1 = 2 * precision * recall / (precision + recall)
    return correct_res_groups, wrong_res_groups

def fetch_inlinks_by_ent(ent):
    cur = conn_psql_kongzi2.cursor()
    sql = "SELECT _id FROM wikipedia_links_2014 WHERE target=%s;"
    cur.execute(sql, (ent,))
    rows = cur.fetchall()
    cur.close()
    return rows

# fetch an entity's outlinks with duplicates
def fetch_outlinks_by_ent(ent):
    cur = conn_psql_kongzi2.cursor()
    sql = "SELECT target FROM wikipedia_links_2014 WHERE _id=%s;"
    cur.execute(sql, (ent,))
    rows = cur.fetchall()
    cur.close()
    return rows

def fetch_entity_by_mention_emnlp17(mention):
    # print(mention)
    cur = conn_psql_kongzi2.cursor()
    # do a PostgreSQL join to select the entity namestring from the tables dictionary and entity_ids
    sql = "SELECT entity, prior FROM men_ent_dict_emnlp2017 WHERE men_ent_dict_emnlp2017.mention = (E\'%s\') ORDER BY prior DESC;"
    cur.execute(sql % mention.replace("'", "\\'"))
    rows = cur.fetchall()
    cur.close()
    return rows

def fetch_inlinks_redis(ent, link_type='inlinks'):
    inlinks = conn.hmget(link_type, ent)[0]
    return json.loads(inlinks) if inlinks else []

def has_inlinks_redis(ent, link_type='inlinks'):
    inlinks = conn.hexists(link_type, ent)
    return inlinks

def save_inlinks_redis(ent, inlinks, link_type='inlinks'):
    conn.hset(link_type, ent, json.dumps(inlinks))
    
def fetch_outlinks_redis(ent, link_type='outlinks'):
    outlinks = conn.hmget(link_type, ent)[0]
    return json.loads(outlinks) if outlinks else []

def has_outlinks_redis(ent, link_type='outlinks'):
    inlinks = conn.hexists(link_type, ent)
    return inlinks

def save_outlinks_redis(ent, outlinks, link_type='outlinks'):
    conn.hset(link_type, ent, json.dumps(outlinks))
    
def check_links_between_ents(ent_1, ent_2, bidirection=False):
    wiki_pre_str = 'en.wikipedia.org/wiki/'
    inlinks_ent_1 = fetch_inlinks_redis(ent_1, link_type='inlinks')
    if not inlinks_ent_1 and not has_inlinks_redis(ent_1):
        print("PostgreSQL: fetching inlinks for entity {}...".format(ent_1))
        wiki_ents_1 = wiki_pre_str + ent_1
        inlinks_ent_1_db = fetch_inlinks_by_ent(wiki_ents_1)
        inlinks_ent_1 = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_1_db]
        print("Redis: caching inlinks for entity {}...".format(ent_1))
        save_inlinks_redis(ent_1, inlinks_ent_1)
    inlinks_ent_2 = fetch_inlinks_redis(ent_2, link_type='inlinks')
    if not inlinks_ent_2 and not has_inlinks_redis(ent_2):        
        wiki_ents_2 = wiki_pre_str + ent_2
        print("PostgreSQL: fetching inlinks for entity {}...".format(ent_2))
        inlinks_ent_2_db = fetch_inlinks_by_ent(wiki_ents_2)
        inlinks_ent_2 = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_2_db]
        print("Redis: caching inlinks for entity {}...".format(ent_2))
        save_inlinks_redis(ent_2, inlinks_ent_2)
    return (ent_1 in inlinks_ent_2 or ent_2 in inlinks_ent_1) if not bidirection else (ent_1 in inlinks_ent_2 and ent_2 in inlinks_ent_1)

def get_links_by_ent(ent, link_type='inlinks'):
    wiki_pre_str = 'en.wikipedia.org/wiki/'
    if link_type == 'inlinks':
        inlinks_ent = fetch_inlinks_redis(ent, link_type='inlinks')
        if not inlinks_ent and not has_inlinks_redis(ent):
#             print("PostgreSQL: fetching inlinks for entity {}...".format(ent))
            wiki_ents = wiki_pre_str + ent
            inlinks_ent_db = fetch_inlinks_by_ent(wiki_ents)
            inlinks_ent = [x[0].replace(wiki_pre_str, '') for x in inlinks_ent_db]
            print("Redis: caching inlinks for entity {}...".format(ent))
            save_inlinks_redis(ent, inlinks_ent)
        return inlinks_ent
    if link_type == 'outlinks':
        outlinks_ent = fetch_outlinks_redis(ent)
        if not outlinks_ent and not has_outlinks_redis(ent):
#             print("PostgreSQL: fetching outlinks for entity {}...".format(ent))
            wiki_ents = wiki_pre_str + ent
            outlinks_ent_db = fetch_outlinks_by_ent(wiki_ents)
            outlinks_ent = [x[0].replace(wiki_pre_str, '') for x in outlinks_ent_db]
            print("Redis: caching outlinks for entity {}...".format(ent))
            save_outlinks_redis(ent, outlinks_ent)
        return outlinks_ent
    
def fetch_ents_by_doc_redis(doc_id):
    id_ents = conn.hmget('doc-predicted-ents-coref-new', doc_id)[0]
    return json.loads(id_ents) if id_ents else []

## Normalized Google Distance
import math
def ngd_similarity(ents_s, ents_t, index_size = 6274625):
    ent_sets_s = set(ents_s)
    ent_sets_t = set(ents_t)
    min_links, max_links = min(len(ent_sets_s), len(ent_sets_t)), max(len(ent_sets_s), len(ent_sets_t))
    com_links = len(ent_sets_s & ent_sets_t)
    if min_links and max_links and com_links:
        return 1 - (math.log(max_links) - math.log(com_links))/ (math.log(index_size) - math.log(min_links))
    else:
        return 0
    
# PMI
def pmi_similarity(ents_s, ents_t, index_size = 6274625, normalize=False):
    ent_sets_s = set(ents_s)
    ent_sets_t = set(ents_t)
    s_links, t_links = len(ent_sets_s), len(ent_sets_t)
    com_links = len(ent_sets_s & ent_sets_t)
    p_s = s_links / index_size
    p_t = t_links / index_size
    p_c = com_links / index_size
    print(p_s, p_t, p_c)
    if p_s and p_t and p_c:
        return p_c/(p_s * p_t) if not normalize else p_c / (p_s * p_t) / min(1/p_s, 1/p_t)
    else:
        return 0

In [8]:
model_dir_path = '.'
def save_model(model, name):
    if not os.path.exists(model_dir_path):
        os.makedirs(model_dir_path)
    model_path = os.path.join(model_dir_path, '%s.mdl' % name)
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
        
def load_model(name):
    model_path = os.path.join(model_dir_path, '%s.mdl' % name)
    with open(model_path, 'rb') as f:
        return pickle.load(f)

In [17]:
def process(process_name ,train_set, testa_set, testb_set, train_total, testa_total, testb_total, 
            n_estimators, max_depths):
    dtrain_xgboost = trans_data(train_set)
    dtest_a_xgboost = trans_data(testa_set)
    dtest_b_xgboost = trans_data(testb_set)

#     train_true_labels = train_set[:, -1].nonzero()[0].size
#     testa_true_labels = testa_set[:, -1].nonzero()[0].size
#     testb_true_labels = testb_set[:, -1].nonzero()[0].size
    
#     print('True labels:', train_true_labels, testa_true_labels, testb_true_labels)
    
    for x in n_estimators:
        num_round = x
        for dep in max_depths:
            model_name = '%d_%d_%s' % (num_round, dep, process_name)
            print(datetime.now(), 'Loading model: %s' % model_name)
            bst = load_model(model_name)

            print(datetime.now(), 'Start evaluation')
            preds = bst.predict(dtrain_xgboost)
            a = evalerror_detail_log(preds, dtrain_xgboost, train_total)
            print("n_estimators: {}, max_depth: {}, acc_training: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            preds = bst.predict(dtest_a_xgboost)
            a = evalerror_detail_log(preds, dtest_a_xgboost, testa_total)
            print("n_estimators: {}, max_depth: {}, acc_validation: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            preds = bst.predict(dtest_b_xgboost)
            a = evalerror_detail_log(preds, dtest_b_xgboost, testb_total)
            print("n_estimators: {}, max_depth: {}, acc_test: {}, corr_num: {}".format(num_round, dep, a[2], a[0]))
            print(datetime.now(), 'Evaluation finished')   

In [18]:
def get_total_mentions(data_source, data_type) -> int:
    with conn_psql.cursor() as cur:
        sql = SQL("select count(*) from {} where annotation != 'NIL' and type=%s").format(Identifier(data_source))
        cur.execute(sql, (data_type,))
        return cur.fetchone()[0]

In [16]:
d_train_total = get_total_mentions('aida_conll', 'train')
d_test_a_total = get_total_mentions('aida_conll', 'testa')
d_test_b_total = get_total_mentions('aida_conll', 'testb')

print(d_train_total, d_test_a_total, d_test_b_total)

18541 4791 4485


In [12]:
d_train_ctx = fetch_all_features('basic_fea_ctx', 'train', 'aida_conll')
d_test_a_ctx = fetch_all_features('basic_fea_ctx', 'testa', 'aida_conll')
d_test_b_ctx = fetch_all_features('basic_fea_ctx', 'testb', 'aida_conll')
d_train_coref = fetch_all_features('basic_fea_coref', 'train', 'aida_conll')
d_test_a_coref = fetch_all_features('basic_fea_coref', 'testa', 'aida_conll')
d_test_b_coref = fetch_all_features('basic_fea_coref', 'testb', 'aida_conll')

In [14]:
print(d_train_ctx.shape)
print(d_test_a_ctx.shape)
print(d_test_b_ctx.shape)
print(d_train_coref.shape)
print(d_test_a_coref.shape)
print(d_test_b_coref.shape)

(531835, 21)
(136259, 21)
(131893, 21)
(531835, 13)
(136259, 13)
(131893, 13)


In [60]:
d_train_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'train', 'aida_conll')
d_test_a_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'testa', 'aida_conll')
d_test_b_ctx_simplified = fetch_all_features('basic_fea_ctx_simplified', 'testb', 'aida_conll')
d_train_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'train', 'aida_conll')
d_test_a_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'testa', 'aida_conll')
d_test_b_coref_simplified = fetch_all_features('basic_fea_coref_simplified', 'testb', 'aida_conll')

# #hotfix
# indices = np.arange(d_train_ctx_simplified.shape[-1])
# indices[2] = 4
# indices[4] = 2
# d_train_ctx_simplified = d_train_ctx_simplified[:, indices]
# d_test_a_ctx_simplified = d_test_a_ctx_simplified[:, indices]
# d_test_b_ctx_simplified = d_test_b_ctx_simplified[:, indices]

print(d_train_ctx_simplified.shape)
print(d_test_a_ctx_simplified.shape)
print(d_test_b_ctx_simplified.shape)
print(d_train_coref_simplified.shape)
print(d_test_a_coref_simplified.shape)
print(d_test_b_coref_simplified.shape)

(531835, 17)
(136259, 17)
(131893, 17)
(531835, 10)
(136259, 10)
(131893, 10)


In [13]:
d_train_ctx_coref  = combine_features(d_train_ctx, d_train_coref)
d_test_a_ctx_coref = combine_features(d_test_a_ctx, d_test_a_coref)
d_test_b_ctx_coref = combine_features(d_test_b_ctx, d_test_b_coref)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [15]:
print(d_train_ctx_coref.shape)
print(d_test_a_ctx_coref.shape)
print(d_test_b_ctx_coref.shape)

(531835, 32)
(136259, 32)
(131893, 32)


In [61]:
d_train_ctx_coref_simplified  = combine_features(d_train_ctx_simplified, d_train_coref_simplified)
d_test_a_ctx_coref_simplified = combine_features(d_test_a_ctx_simplified, d_test_a_coref_simplified)
d_test_b_ctx_coref_simplified = combine_features(d_test_b_ctx_simplified, d_test_b_coref_simplified)
print(d_train_ctx_coref_simplified.shape)
print(d_test_a_ctx_coref_simplified.shape)
print(d_test_b_ctx_coref_simplified.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 25)
(136259, 25)
(131893, 25)


In [24]:
d_train_coh = fetch_all_features('basic_fea_coh', 'train', 'aida_conll')
d_test_a_coh = fetch_all_features('basic_fea_coh', 'testa', 'aida_conll')
d_test_b_coh = fetch_all_features('basic_fea_coh', 'testb', 'aida_conll')

In [25]:
print(d_train_coh.shape)
print(d_test_a_coh.shape)
print(d_test_b_coh.shape)

(531835, 24)
(136259, 24)
(131893, 24)


In [26]:
d_train_ctx_coref_coh  = combine_features(d_train_ctx_coref, d_train_coh)
d_test_a_ctx_coref_coh = combine_features(d_test_a_ctx_coref, d_test_a_coh)
d_test_b_ctx_coref_coh = combine_features(d_test_b_ctx_coref, d_test_b_coh)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...


In [27]:
print(d_train_ctx_coref_coh.shape)
print(d_test_a_ctx_coref_coh.shape)
print(d_test_b_ctx_coref_coh.shape)

(531835, 54)
(136259, 54)
(131893, 54)


In [62]:
d_train_ctx_coref_coh_simplified  = combine_features(d_train_ctx_coref_simplified, d_train_coh)
d_test_a_ctx_coref_coh_simplified = combine_features(d_test_a_ctx_coref_simplified, d_test_a_coh)
d_test_b_ctx_coref_coh_simplified = combine_features(d_test_b_ctx_coref_simplified, d_test_b_coh)
print(d_train_ctx_coref_coh_simplified.shape)
print(d_test_a_ctx_coref_coh_simplified.shape)
print(d_test_b_ctx_coref_coh_simplified.shape)

Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
Building idx for new features...
Combine original and new features...
(531835, 47)
(136259, 47)
(131893, 47)


In [36]:
d_train_ctx_no_poly = fetch_all_features('basic_no_poly', 'train', 'aida_conll')
d_test_a_ctx_no_poly = fetch_all_features('basic_no_poly', 'testa', 'aida_conll')
d_test_b_ctx_no_poly = fetch_all_features('basic_no_poly', 'testb', 'aida_conll')
print(d_train_ctx_no_poly.shape)
print(d_test_a_ctx_no_poly.shape)
print(d_test_b_ctx_no_poly.shape)

(531835, 23)
(136259, 23)
(131893, 23)


In [37]:
d_train_ctx_poly = fetch_all_features('basic_poly', 'train', 'aida_conll')
d_test_a_ctx_poly = fetch_all_features('basic_poly', 'testa', 'aida_conll')
d_test_b_ctx_poly = fetch_all_features('basic_poly', 'testb', 'aida_conll')
print(d_train_ctx_poly.shape)
print(d_test_a_ctx_poly.shape)
print(d_test_b_ctx_poly.shape)

(531835, 17)
(136259, 17)
(131893, 17)


In [65]:
process('coref_new', d_train_ctx_coref_simplified, d_test_a_ctx_coref_simplified, d_test_b_ctx_coref_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-17 22:16:56.416965 Loading model: 4900_6_coref_new
2019-08-17 22:16:56.443034 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.782589935817917, corr_num: 14510
n_estimators: 4900, max_depth: 6, acc_validation: 0.7422250052181173, corr_num: 3556
n_estimators: 4900, max_depth: 6, acc_test: 0.7547380156075808, corr_num: 3385
2019-08-17 22:17:22.063862 Evaluation finished


In [21]:
process('coref_new_emnlp17', d_train_ctx_coref, d_test_a_ctx_coref, d_test_b_ctx_coref, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 14:17:49.062497 Loading model: 4900_6_coref_new_emnlp17
2019-08-16 14:17:49.098575 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.8564802329971415, corr_num: 15880
n_estimators: 4900, max_depth: 6, acc_validation: 0.8102692548528491, corr_num: 3882
n_estimators: 4900, max_depth: 6, acc_test: 0.7933110367892977, corr_num: 3558
2019-08-16 14:18:11.543792 Evaluation finished


In [66]:
process('coref_new_emnlp17_for_review', 
        d_train_ctx_coref_simplified, d_test_a_ctx_coref_simplified, d_test_b_ctx_coref_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-17 22:17:22.277329 Loading model: 4900_6_coref_new_emnlp17_for_review
2019-08-17 22:17:22.307015 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.7369073944231703, corr_num: 13663
n_estimators: 4900, max_depth: 6, acc_validation: 0.7006887914840325, corr_num: 3357
n_estimators: 4900, max_depth: 6, acc_test: 0.711705685618729, corr_num: 3192
2019-08-17 22:17:45.479320 Evaluation finished


In [67]:
process('coref_new_coh', 
        d_train_ctx_coref_coh_simplified, d_test_a_ctx_coref_coh_simplified, d_test_b_ctx_coref_coh_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-17 22:17:45.899298 Loading model: 4900_6_coref_new_coh
2019-08-17 22:17:45.928679 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.8916994768351222, corr_num: 16533
n_estimators: 4900, max_depth: 6, acc_validation: 0.8515967438948028, corr_num: 4080
n_estimators: 4900, max_depth: 6, acc_test: 0.853288740245262, corr_num: 3827
2019-08-17 22:18:08.045637 Evaluation finished


In [29]:
process('coref_new_coh_emnlp17', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 14:22:41.290249 Loading model: 4900_6_coref_new_coh_emnlp17
2019-08-16 14:22:41.319749 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.9381371015587078, corr_num: 17394
n_estimators: 4900, max_depth: 6, acc_validation: 0.8712168649551242, corr_num: 4174
n_estimators: 4900, max_depth: 6, acc_test: 0.875139353400223, corr_num: 3925
2019-08-16 14:23:01.949405 Evaluation finished


In [30]:
process('coref_new_coh_emnlp17_for_review', d_train_ctx_coref_coh, d_test_a_ctx_coref_coh, d_test_b_ctx_coref_coh, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-16 14:23:02.382560 Loading model: 4900_6_coref_new_coh_emnlp17_for_review
2019-08-16 14:23:02.414279 Start evaluation


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51']
training data did not have the following fields: f49, f48, f50, f47, f51, f42, f44, f45, f43, f46

In [68]:
process('no_poly', d_train_ctx_simplified, d_test_a_ctx_simplified, d_test_b_ctx_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

2019-08-17 22:18:08.243232 Loading model: 4900_6_no_poly
2019-08-17 22:18:08.279071 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.7490965967315679, corr_num: 13889
n_estimators: 4900, max_depth: 6, acc_validation: 0.7278230014610728, corr_num: 3487
n_estimators: 4900, max_depth: 6, acc_test: 0.7228539576365663, corr_num: 3242
2019-08-17 22:18:32.961397 Evaluation finished


In [69]:
process('no_poly', d_train_ctx_simplified, d_test_a_ctx_simplified, d_test_b_ctx_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[1700], max_depths=[8])

2019-08-17 22:18:33.116809 Loading model: 1700_8_no_poly
2019-08-17 22:18:33.152419 Start evaluation
n_estimators: 1700, max_depth: 8, acc_training: 0.743109864624346, corr_num: 13778
n_estimators: 1700, max_depth: 8, acc_validation: 0.723022333542058, corr_num: 3464
n_estimators: 1700, max_depth: 8, acc_test: 0.7072463768115942, corr_num: 3172
2019-08-17 22:18:44.796798 Evaluation finished


In [70]:
print(d_train_ctx_coref_coh.shape)
process('global', 
        d_train_ctx_coref_coh_simplified, d_test_a_ctx_coref_coh_simplified, d_test_b_ctx_coref_coh_simplified, 
        d_train_total, d_test_a_total, d_test_b_total,
        n_estimators=[4900], max_depths=[6])

(531835, 54)
2019-08-17 22:18:45.249010 Loading model: 4900_6_global
2019-08-17 22:18:45.275783 Start evaluation
n_estimators: 4900, max_depth: 6, acc_training: 0.8781619114395124, corr_num: 16282
n_estimators: 4900, max_depth: 6, acc_validation: 0.8265497808390733, corr_num: 3960
n_estimators: 4900, max_depth: 6, acc_test: 0.8390189520624304, corr_num: 3763
2019-08-17 22:19:06.947047 Evaluation finished
